# Uniswap EDA

## Imports

In [1]:
# <include-uniswap_eda/utils.py>

In [2]:
# <imports>
from datetime import datetime

import pandas as pd
import plotly.io as pio

from uniswap_eda import utils

pd.options.plotting.backend = "plotly"
pio.templates.default = "none"

from sgqlc.operation import Operation
from schema import schema
from sgqlc.endpoint.http import HTTPEndpoint

url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'

endpoint = HTTPEndpoint(url)

## Overview


This notebook is an exploration of how Uniswap-V3 works. My basic understanding at this point is that a pool consist of liquidity providers who add pairs of tokens to a pool to be exchanged in what are called swaps by users. Liquidity providers share the fees that are generated from users swapping tokens. The price at which tokens are swapped is determined by a constant product formula. The fees that are earned are distributed based on the amount of a liquidity that each provider provided for the amount of time that the price was between the ticks that liquidity provider was providing liquidity in.

### How is the uniswap price determined?

By the constant product formula with a slippage percentage specified relative to a price from an off chain oracle.

Get historical twap pricing from uniswap api and compare to pricing from 

##

## Resources


* [Help Center](https://help.uniswap.org/en/)
* [Documentation](https://docs.uniswap.org/)
* [Github](https://github.com/Uniswap/uniswap-v3-core)
* [info.uniswap.org](https://info.uniswap.org)
* [White Paper](https://uniswap.org/whitepaper-v3.pdf)
* [Discord Channel](https://discord.com/channels/597638925346930701/597638926152499206)
* [Strategy Simulator](https://defi-lab.xyz/uniswapv3simulator)
* [Uniswap Prices Calculation](https://github.com/thanpolas/univ3prices)
* [Subgraph Schema](https://github.com/Uniswap/uniswap-v3-subgraph/blob/main/schema.graphql)
* [RFPs and Challenges](https://www.notion.so/RFPs-Challenges-3be614ba4e504b5caeee7b0159e64a42)
* [Ethereum Strategies](https://twitter.com/_jamiis/status/1360322712414076931)

## Exploration of a Single Pool

The GraphQL query below gets the largest v3 pool by total value locked in USD terms. This happens to be USD Coin (USDC) versus Wrapped Ether (WETH) (which is just original ether that was introduced before the ERC-20 standard, wrapped with an ERC-20 smart contract). This pool was created on 2021-05-04. [Here](https://info.uniswap.org/#/pools/0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8) is the link to the pool stats on info.uniswap.org. That site says as of 12:42 pacific on 2021-07-21 that total value locked was $252.88 million, which corresponds pretty closely to the value of 253801793.24 per `totalValueLockedUSD`. `totalValueLockedtoken0` and `totalValueLockedtoken1` are also quite similar.

The fee amount for this pool is 0.3% and a total of $56,278,508 (`feesUSD`) in fees have been collected to date. The total volume (`volumeUSD`) in USD to date is $18,759,502,936, which is exactly 0.003. `feeTier` is in thousands to needs to get divided by 1e6 to get to the floating point representation.

Tick spacing can be 10, 50 or 200, associated with the fee tiers for 0.05%, 0.3% and 1% see [subgraph](https://github.com/Uniswap/uniswap-v3-subgraph/blob/fd64afaca4926a476896c993c6f9ff895bf986c5/src/utils/tick.ts#L43).

Things to be able to do:

* Explore the Position data
* Calculate liquidity for a tick from positions
* Equate pool liquidity with tick liquidity for current tick
* Explore functions for 
* Calculate time weighed average price from two accumulator values
* Add latex formulas to tie out to python functions / results
* Candlestick chart for prices

![USDC WETH Overview](usdc_weth_2021-07-21_12-43-15.png)

In [21]:
op = Operation(schema.Query)
pool = op.pool(id="0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8")
pool.__fields__(
    mints=False,
    burns=False,
    swaps=False,
    pool_hour_data=False,
    pool_day_data=False,
    ticks=False,
    collects=False
)
pool.token0.__fields__(
    symbol=True,
    name=True,
    decimals=True,
    total_supply=True
)
pool.token1.__fields__(
    symbol=True,
    name=True,
    decimals=True,
    total_supply=True
)
# pool_hour_data = pools.pool_hour_data(first=5, order_by="periodStartUnix", order_direction="desc").__fields__(pool=False)
# pool.ticks(first=5, order_by="tickIdx", order_direction="desc", where={"tick_idx_lt": -47546}).__fields__(pool=False)

pool = endpoint(op)["data"]["pool"]

In [23]:
pool

{'collectedFeesToken0': '0',
 'collectedFeesToken1': '0',
 'collectedFeesUSD': '0',
 'createdAtBlockNumber': '12370624',
 'createdAtTimestamp': '1620169800',
 'feeGrowthGlobal0X128': '879608981722641786094544848009357',
 'feeGrowthGlobal1X128': '352367575245191929105225043689586084123335',
 'feeTier': '3000',
 'feesUSD': '56765567.63281827206766270036625933',
 'id': '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8',
 'liquidity': '17846142292882053102',
 'liquidityProviderCount': '0',
 'observationIndex': '0',
 'sqrtPrice': '1744634243657861946041054518206014',
 'tick': '200004',
 'token0': {'decimals': '6',
  'name': 'USD Coin',
  'symbol': 'USDC',
  'totalSupply': '19312'},
 'token0Price': '2062.293069916995817242368269060948',
 'token1': {'decimals': '18',
  'name': 'Wrapped Ether',
  'symbol': 'WETH',
  'totalSupply': '19848'},
 'token1Price': '0.0004848971344505601671670196813582106',
 'totalValueLockedETH': '127096.7459814601576682515930193351',
 'totalValueLockedToken0': '112291780.9

In [4]:
createdAt = datetime.fromtimestamp(int(pool["createdAtTimestamp"])).isoformat()
feePct = float(pool["feesUSD"]) / float(pool["volumeUSD"])
3000 / 1e6

0.003

## Position

In [79]:
op = Operation(schema.Query)
positions = op.positions(first=10, where={"pool": "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8"})

positions.tick_lower(id="0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8")

positions.__fields__(
    pool=False,
    token0=False,
    token1=False,
    mints=False,
    burns=False,
    collects=False,
    swaps=False,
    tick_lower=False
)


# positions.tick_upper(where={"tickIdx_gte": "200004"})

# tick_lowers.__fields__(
#     pool=False
# )

# tick_uppers.__fields__(
#     pool=False
# )
endpoint(op)
# op

KeyError: 'id'

In [34]:
positions

[{'collectedFeesToken0': '0',
  'collectedFeesToken1': '0',
  'depositedToken0': '11',
  'depositedToken1': '0.006256923560032086',
  'feeGrowthInside0LastX128': '11051946651142285490120434851970',
  'feeGrowthInside1LastX128': '2853664887402123112825648144657839284339',
  'id': '10023',
  'liquidity': '395460364753',
  'owner': '0xafbad32c97e97075587566a691c5b613db35a596',
  'tickLower': {'collectedFeesToken0': '0',
   'collectedFeesToken1': '0',
   'collectedFeesUSD': '0',
   'createdAtBlockNumber': '12379317',
   'createdAtTimestamp': '1620284794',
   'feeGrowthOutside0X128': '7754172485245761590668832257383',
   'feeGrowthOutside1X128': '1985458784020810623645127911188459111286',
   'feesUSD': '0',
   'id': '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8#138180',
   'liquidityGross': '5993742055125482',
   'liquidityNet': '5993742055125482',
   'liquidityProviderCount': '0',
   'poolAddress': '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8',
   'price0': '1001800.20652198519223578233294970

## Transactions

In [34]:
op = Operation(schema.Query)
candles = op.transactions(where={"pool": "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8", "timestamp_lte": 1621810558}, order_by="timestamp", order_direction="desc", first=10)
# candles = op.transactions(where={"pool": "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8"}, order_direction="desc", first=10, skip=5000)
candles.__fields__(
    id=True,
    timestamp=True
    # high=True,
    # low=True,
    # close=True
)

# data = endpoint(op)
# len(data["data"]["transactions"])
endpoint(op)

{'data': {'transactions': [{'id': '0x3db003c57ca942913c3a5616ff14595c5f227b3b4b873e9ec46a80c68b249b8e',
    'timestamp': '1621810556'},
   {'id': '0xd6596fe0d76eb3f5c7e7661c7cf94f388ce3936ff8276e1825e4362c7faccfa1',
    'timestamp': '1621810536'},
   {'id': '0xc9e279af85e5554cd0cd9a8035335a3145860789475451471fa96ff6d1454564',
    'timestamp': '1621810536'},
   {'id': '0x9d99dc432462a0e29c792a9684160b2e0c5935d3104055f1a0b61f80579ef026',
    'timestamp': '1621810536'},
   {'id': '0x6541e62e6bfd529b4d36e31f05f153445bb7a108f3a154192ab1490618dc7009',
    'timestamp': '1621810536'},
   {'id': '0x33576ef5b5fb928be6e5bbf0ee56b974919d5668c60ea0b6d327d7b632f0f12c',
    'timestamp': '1621810536'},
   {'id': '0x1542197d71a0ca5574380661583f9554abe21573308067475f48814233e47966',
    'timestamp': '1621810536'},
   {'id': '0xbc99f2c5b914d08f37b25df50b7f8a7707f135d32d4cf41bfe73afb8f0526310',
    'timestamp': '1621810535'},
   {'id': '0xb639cb319d9f619e2ca0ec7dfaa705fa7681d6b8298a379c1b824997ad28ffdf',


## Top Pools

In [1]:
op = Operation(schema.Query)
pools = op.pools(order_by="totalValueLockedUSD", order_direction="desc", first=10)
pools.__fields__(
    mints=False,
    burns=False,
    swaps=False,
    pool_hour_data=False,
    pool_day_data=False,
    ticks=False,
    collects=False
)
pools.token0.__fields__(
    symbol=True,
    name=True,
    decimals=True,
    total_supply=True
)
pools.token1.__fields__(
    symbol=True,
    name=True,
    decimals=True,
    total_supply=True
)
# pool_hour_data = pools.pool_hour_data(first=5, order_by="periodStartUnix", order_direction="desc").__fields__(pool=False)
# pools.ticks(first=5, order_by="tickIdx", order_direction="desc", where={"tick_idx_lt": 200401}).__fields__(pool=False)
data = endpoint(op)
pools = data["data"]["pools"]

NameError: name 'Operation' is not defined

In [18]:
pool_recs = []
for p in pools:
    pool_rec = {}
    pool_rec["id"] = p["id"]
    pool_rec["token0"] = p["token0"]["symbol"]
    pool_rec["token1"] = p["token1"]["symbol"]
    pool_rec["tvlUSD_MM"] = round(float(p["totalValueLockedUSD"]) / 1e6,1)
    pool_rec["tok0Dec"] = int(p["token0"]["decimals"])
    pool_rec["tok11Dec"] = int(p["token1"]["decimals"])
    pool_rec["token1Price"] = round(float(p["token1Price"]), 6)
    pool_rec["token0Price"] = round(float(p["token0Price"]), 6)
    
    pool_rec["sqrtPrice"] = int(p["sqrtPrice"])
    pool_rec["tok1Price_sqrtPrice"] = utils.sqrt_price_to_tick(p["sqrtPrice"], p["token0"]["decimals"], p["token1"]["decimals"]) 
    pool_rec["tok0Price_sqrtPrice"] = 1 / utils.sqrt_price_to_tick(p["sqrtPrice"], p["token0"]["decimals"], p["token1"]["decimals"])
    pool_rec["tick"] = int(p["tick"])
    pool_rec["token1Price_tick"] = utils.tick_to_price(p["tick"], p["token0"]["decimals"], p["token1"]["decimals"])
    pool_rec["token0Price_tick"] = 1 / utils.tick_to_price(p["tick"], p["token0"]["decimals"], p["token1"]["decimals"])
    pool_recs.append(pool_rec)
pd.DataFrame(pool_recs)

,id,token0,token1,tvlUSD_MM,tok0Dec,tok11Dec,token1Price,token0Price,sqrtPrice,tok1Price_sqrtPrice,tok0Price_sqrtPrice,tick,token1Price_tick,token0Price_tick
0,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,USDC,WETH,262.6,6,18,0.000484,2066.255632,1742960554604975746812629187750013,0.000484,2066.255632,199985,0.000484,2066.312660
1,0x7858e59e0c01ea06df3af3d20ac7b0003275d4bf,USDC,USDT,188.8,6,6,1.000200,0.999800,79236081571290040430959097622,1.000200,0.999800,1,1.000100,0.999900
2,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC,WETH,149.6,8,18,15.855103,0.063071,31547439728262583459360356042537943,15.855103,0.063071,257906,15.854599,0.063073
3,0x6c6bc977e13df9b0de53b251522280bb72383700,DAI,USDC,117.8,18,6,1.000634,0.999366,79253279332169081581262,1.000634,0.999366,-276318,1.000603,0.999398
4,0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,WETH,USDT,112.8,18,6,2064.881344,0.000484,3600204197638584562734859,2064.881344,0.000484,-199992,2064.866820,0.000484
5,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,WETH,107.3,6,18,0.000483,2068.881685,1741854024512922358092938238159909,0.000483,2068.881685,199972,0.000483,2069.000479
6,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801,UNI,WETH,57.4,18,18,0.008699,114.958410,7389400580308568819619073728,0.008699,114.958410,-47449,0.008698,114.969028
7,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,WBTC,USDC,53.7,8,6,32675.404388,0.000031,1432155150780408377639915877657,32675.404388,0.000031,57894,32672.229173,0.000031
8,0x6f48eca74b38d2936b02ab603ff4e36a6c0e3a77,DAI,USDT,34.8,18,6,1.000645,0.999355,79253712331600947372848,1.000645,0.999355,-276318,1.000603,0.999398
9,0x8f8ef111b67c04eb1641f5ff19ee54cda062f163,WBTC,PAX,34.5,8,18,32669.670987,0.000031,1432029498464517166380285248000000000,32669.670987,0.000031,334217,32668.875905,0.000031


In [141]:
accumulator_delta = float(pool["ticks"][0]["price0"]) - float(pool["ticks"][1]["price0"])
accumulator_delta

3017544.2882022858

In [156]:
P = float(pool["totalValueLockedToken0"]) / float(pool["totalValueLockedToken1"])
P

1263.6335317133787

In [142]:
1.0001 ** (accumulator_delta / (60 * 60))

1.0874293147290275

In [145]:
float(pool["totalValueLockedToken0"]) * float(pool["totalValueLockedToken1"])

7822359097369.466

In [59]:
float_cols = ["open", "high", "low", "close", "tvlUSD", "token0Price", "token1Price"]
int_cols = []
df_hourly = pd.DataFrame(pool["poolHourData"]).astype({fc: float for fc in float_cols})
df_hourly['dateTime'] = pd.to_datetime(df_hourly['periodStartUnix'],unit='s')
df_hourly

,close,feeGrowthGlobal0X128,feeGrowthGlobal1X128,feesUSD,high,id,liquidity,low,open,periodStartUnix,sqrtPrice,tick,token0Price,token1Price,tvlUSD,txCount,volumeToken0,volumeToken1,volumeUSD,dateTime
0,1977.973473,874979102996300954639254533655225,350526631139346628401877253647519048631880,0,1980.551230,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8-451919,16827547024249704226,1977.623948,1977.623948,1626908400,1781432422475807032952268090297621,200421,1977.973473,0.000506,2.561168e+08,16,0,0,0,2021-07-21 23:00:00
1,1977.623948,874945415587848166303247496984956,350511643866989506797553816068686360831201,0,1977.623948,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8-451918,16798546947525575029,1955.510694,1955.581237,1626904800,1781589840477654114820610143114369,200423,1977.623948,0.000506,2.558838e+08,59,0,0,0,2021-07-21 22:00:00
2,1955.865401,874690117570771161772367726492384,350509515883741698380136440683059205716783,0,1957.761480,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8-451917,14869030150748591690,1945.387827,1945.387827,1626901200,1791472317473467759980697920259631,200534,1955.865401,0.000511,2.541562e+08,35,0,0,0,2021-07-21 21:00:00
3,1945.387827,874545958152337591452542879165510,350498326407533841641649542080782446460676,0,1945.387827,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8-451916,14737784200884702323,1938.133172,1942.339835,1626897600,1796290142931039666189921906336321,200588,1945.387827,0.000514,2.525898e+08,57,0,0,0,2021-07-21 20:00:00
4,1943.737562,874437759799881774259209905800226,350452060049857228339394360828448927565015,0,1963.717245,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8-451915,14660072282118470820,1943.737562,1963.127285,1626894000,1797052520925717111937140078842364,200596,1943.737562,0.000514,2.520735e+08,63,0,0,0,2021-07-21 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1942.968347,847645186763524894517975186999431,336283888172957314616112121397072447415417,0,1942.968347,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8-451724,13306754957595312003,1923.488703,1923.488703,1626206400,1797408209772151303734129377873357,200600,1942.968347,0.000515,2.639464e+08,44,0,0,0,2021-07-13 20:00:00
196,1926.517344,847418369384670465273104138021348,336264742589107028197855709318325633384793,0,1965.722094,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8-451723,12969373854580066017,1925.028308,1965.722094,1626202800,1805066150465717816513402848358857,200685,1926.517344,0.000519,2.638753e+08,83,0,0,0,2021-07-13 19:00:00
197,1966.409817,847331376612188759244029710822828,335982848589224712851109510334043509555556,0,1991.830363,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8-451722,17404347783651843722,1966.409817,1991.830363,1626199200,1786662684228086127112999931843535,200480,1966.409817,0.000509,2.676258e+08,103,0,0,0,2021-07-13 18:00:00
198,1991.831065,847331094630383847068287836131264,335834885553102697296478130920047728823840,0,1992.428009,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8-451721,17117872078150877688,1991.831065,1992.428009,1626195600,1775224704455328628136713983082123,200352,1991.831065,0.000502,2.700932e+08,27,0,0,0,2021-07-13 17:00:00


## State

Tracked at the following levels:

* global - per pool
* per-tick
* per-position